# Chatbot with history

In this video We'll go over an example, using Groq, of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f32c458a230>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f32c45c48e0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hi Krish, it's nice to meet you!  \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days? \n\nI'm always eager to learn more about the cutting-edge applications of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.00018781, 'queue_time': 0.013450609, 'total_time': 0.102005992}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-56f5c96a-6da2-4095-b4b7-6367f85e8b21-0', usage_metadata={'input_tokens': 22, 'output_tokens': 56, 'total_tokens': 78})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 97, 'total_tokens': 139, 'completion_time': 0.076363636, 'prompt_time': 0.002554412, 'queue_time': 0.010738194999999999, 'total_time': 0.078918048}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e28be65b-bbd9-4876-8ef9-ce411a8ac4a2-0', usage_metadata={'input_tokens': 97, 'output_tokens': 42, 'total_tokens': 139})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} # dictionary

# each conversation is a sessions, and has its own unique session_id
# -> return type is BaseChatMessageHistory
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)
response.content

"Hello Krish! It's great to meet you.  Being a Chief AI Engineer is a really exciting role. \n\nWhat kind of projects are you working on these days?  I'm always interested in hearing about the innovative things people are doing with AI.  \n\nPerhaps you could tell me:\n\n* **What industry are you in?**\n* **What are some of the biggest challenges you face as a Chief AI Engineer?**\n* **What are you most excited about for the future of AI?** \n\n\nI'm eager to learn more! 😊  \n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish!  \n\nI remember from our previous conversation. 😊  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 417, 'total_tokens': 437, 'completion_time': 0.036363636, 'prompt_time': 0.013323565, 'queue_time': 0.0027569740000000006, 'total_time': 0.049687201}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b20f1b17-2ae1-4af2-8f9b-54c605c94eb6-0', usage_metadata={'input_tokens': 417, 'output_tokens': 20, 'total_tokens': 437})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John!  It's nice to meet you. \n\nWhat can I do for you today?\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nIs there anything else I can help you with, John?  😄  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! It's nice to meet you. \n\nHow can I help you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 31, 'total_tokens': 56, 'completion_time': 0.045454545, 'prompt_time': 0.000375719, 'queue_time': 0.012927292, 'total_time': 0.045830264}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c46e64b9-becd-488c-a493-b39c31dfe33e-0', usage_metadata={'input_tokens': 31, 'output_tokens': 25, 'total_tokens': 56})

In [19]:
chain.invoke({"messages":[HumanMessage(content="What's my name?")]})

AIMessage(content="As an AI, I have no memory of past conversations and don't know your name.  \n\nCould you tell me your name so I can address you properly? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 32, 'total_tokens': 72, 'completion_time': 0.072727273, 'prompt_time': 0.000346489, 'queue_time': 0.01326105, 'total_time': 0.073073762}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-99ff577e-ea5a-4f5c-83df-2b76683bf102-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [20]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish! \n\nIt's nice to meet you.  What can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 14, 'total_tokens': 42, 'completion_time': 0.050909091, 'prompt_time': 0.00022871, 'queue_time': 0.054381309, 'total_time': 0.051137801}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bca4fc85-28f1-40b1-8a1c-3f83f338d0cd-0', usage_metadata={'input_tokens': 14, 'output_tokens': 28, 'total_tokens': 42})

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  \n\nI remember that you told me.  \n\nIs there anything else I can help you with? 😄 \n'

In [22]:
## Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Italian"})
response.content

'Ciao Krish, benvenuto!  Piacere di conoscerti.  \n\nCome posso aiutarti?\n'

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Italian"},
    config=config
)
repsonse.content

"Ciao Krish!  👋  So che ti chiami Krish, lo hai detto più volte! 😊 \n\nC'è qualcosa in particolare che posso fare per te oggi? \n"

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Italian"},
    config=config,
)
response.content

"Il tuo nome è Krish!  😊  \n\nTi piace ripetere il tuo nome?  😄  \n\nC'è qualcosa che posso fare per te oltre a ripetere il tuo nome? 🤔\n\n\n\n"

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages

# Strategy is to clear the old history

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)
# "hi! I'm bob" has been trimmed (we setup 45 tokens)

In [37]:
from operator import itemgetter # allow to get previous messages based on the trimmer

from langchain_core.runnables import RunnablePassthrough

chain=(
    # RunnablePassthrough is needed to use the trimmer
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content
# it's not able to remember because the 45 tokens are applied after my question

"As an AI, I don't have taste buds or personal preferences like ice cream flavors! \n\nWhat's your favorite flavor? 🍦😋  \n"

In [38]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content
# it's able to remember because the 45 tokens are keeping that statement I did previously

'You asked "what is 2 + 2".  \n\nIs there another problem you\'d like to solve? 😊  \n'

In [41]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain, # chain is trimming the context
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [42]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations and don't know your name.  \n\nWould you like to tell me your name?  😊  \n"

In [43]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  \n\nIf you'd like to ask me a math problem, I'm happy to help! Just let me know what it is. 😊 \n"

In [46]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="I am Paolo")],
        "language": "English",
    },
    config=config,
)

response.content

'Nice to meet you, Paolo! \n\nIs there anything I can help you with? 😊\n'

In [48]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What's my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"